In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sksurv.metrics import concordance_index_censored

# Load datasets
train = pd.read_csv(
    "/kaggle/input/preprocessed-train-and-test-data/preprocessed_train.csv")
test = pd.read_csv(
    "/kaggle/input/preprocessed-train-and-test-data/preprocessed_test.csv")

# Inspect data
print(train.head())
print(test.head())


ModuleNotFoundError: No module named 'sksurv'

In [7]:
# Example: Define complications based on a threshold of a medical metric
# Replace 'medical_metric' with an appropriate column name
train['complications'] = (train['medical_metric'] > threshold).astype(int)

# Drop unnecessary columns
X = train.drop(columns=['ID', 'complications', 'efs', 'efs_time'])
y = train['complications']


NameError: name 'train' is not defined

In [ ]:
# Handle missing values, encoding, and scaling
from sklearn.preprocessing import StandardScaler

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# Train XGBoost model
xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror", n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict risk scores
y_pred = xgb_model.predict(X_val)


In [ ]:
# Example metric: Concordance Index (adapt for binary complications prediction)
c_index = concordance_index_censored(
    event_indicator=y_val,  # Modify this depending on the metric
    event_time=y_val.index,  # Replace with a relevant time-based metric
    estimate=y_pred
)

print(f"Concordance Index: {c_index[0]}")


In [ ]:
# Preprocess test data
X_test = test.drop(columns=['ID'])
X_test_scaled = scaler.transform(X_test)

# Predict complications risk scores
test_predictions = xgb_model.predict(X_test_scaled)

# Prepare submission file
submission = pd.DataFrame({'ID': test['ID'], 'prediction': test_predictions})
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")
